In [1]:
from sklearn.datasets import fetch_20newsgroups
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from collections import defaultdict
import gensim
from InitKmeans import InitKmeans
import numpy as np
import pandas as pd
from sklearn.mixture import GaussianMixture
     
def pre_processData(newsgroups_train):
    tokenizer = RegexpTokenizer(r'\w+')
    for i in range(len(newsgroups_train)):
        newsgroups_train[i] = newsgroups_train[i].lower()
        #newsgroups_train[i] = tokenizer.tokenize(newsgroups_train[i])
    #newsgroups_train = [[token for token in doc if not token.isdigit()] for doc in newsgroups_train]
    newsgroups_train = [doc.split(' ') for doc in newsgroups_train]
    return newsgroups_train
from sklearn.datasets import fetch_20newsgroups
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from collections import defaultdict
import gensim
from InitKmeans import InitKmeans
import numpy as np
import pandas as pd
from sklearn.mixture import GaussianMixture
import sys  
import json
if __name__ == "__main__":
    newsgroups_train = list()
    news_labels = {}
    docID_username = {}
    
    i = 0
    file_path="D:/part_news4_biterm.txt"
    with open(file_path) as fp:
        lines = fp.read().split("\n")
        for line in lines:
            if line:
                text = json.loads(line)["bitermText"].strip()
                label = json.loads(line)["clusterNo"]
                newsgroups_train.append(text)
                news_labels[i] = label
                i+=1
    fp.close()
    
    corpus = pre_processData(newsgroups_train)

D:\anacoda\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
doc_biterm = {}
for docID in range(len(corpus)):
    words = corpus[docID]
    duplicate_biterm = []
    doc_biterm[docID] = {}
    for biterm in words:
        hash_biterm = set(biterm.split(','))
        if hash_biterm not in duplicate_biterm:
            duplicate_biterm.append(hash_biterm)
            doc_biterm[docID][biterm] = 1
        else:
            if biterm in doc_biterm[docID].keys():
                doc_biterm[docID][biterm] += 1
            else:
                doc_biterm[docID][biterm] = 1
doc_words = {}
i_biterm = 0
for docID in range(len(corpus)):
    doc_words[docID] = {}
    words = corpus[docID]
    for biterm in words:
        for word in biterm.split(','):
            if word not in doc_words[docID].keys():
                doc_words[docID][word] = 0
            doc_words[docID][word] += 1

In [22]:
total_batch = 16
NMI_batch = {}
start = 0
end = 0
topic_batch = {}
for batch_i in range(1,17):
    if batch_i == 16:
        end = int(len(corpus))
    else:
        end = int(len(corpus) / total_batch) * batch_i
    
    docID_assign_z = {}
    m_z = {}
    n_z = {}
    n_w = {}
    n_b = {}
    Topics = []
    V = set()
    D = set()
    alpha = 0.3
    beta = 0.002
    NMI_batch[batch_i] = 0
    topic_batch[batch_i] = 0
    
    total_iter = 10
    for iter in range(total_iter):
        for docID in range(start,end):
            words = corpus[docID]
            D.discard(docID)
            if docID in docID_assign_z.keys():
                before_k = docID_assign_z[docID]
                m_z[before_k].discard(docID)
                for biterm in words:
                    for word in biterm.split(','):
                        n_z[before_k][word] -= 1
                        n_w[before_k] -=1
            else:
                before_k = -1
            if len(D) == 0 and len(V) == 0:
                choose_k = 0
                D.add(docID)
                docID_assign_z[docID] = choose_k
                if choose_k not in m_z.keys():
                    m_z[choose_k] = set()
                m_z[choose_k].add(docID)
                for biterm in words:
                    for word in biterm.split(','):
                        if choose_k not in n_w.keys():
                            n_w[choose_k] = 0
                        if choose_k not in n_z.keys():
                            n_z[choose_k] = {}
                        if word not in n_z[choose_k].keys():
                            n_z[choose_k][word] = 0
                        n_z[choose_k][word] += 1
                        n_w[choose_k] += 1
                        V.add(word)
                if choose_k not in Topics:
                    Topics.append(choose_k)
            else:
                log_pro = []
                for k in Topics:
                    pro_k = len(m_z[k])
                    if pro_k != 0:
                        i = 0
                        for biterm in words:
                            each_word = biterm.strip().split(',')
                            for word in each_word:
                                if word not in n_z[k].keys():
                                    n_z[k][word] = 0
                            for j in range(doc_biterm[docID][biterm]):
                                pro_k *= (n_z[k][each_word[0]] + n_z[k][each_word[-1]] + beta + j) / ( n_w[k] + len(V)*beta + i) 
                                i += 1

                    if pro_k == 0:
                        pro_k = sys.float_info.min
                    log_pro.append(pro_k)

                pro_new_k = alpha*(len(D))
                i = 0
                for biterm in words:
                    for j in range(doc_biterm[docID][biterm]):
                        pro_new_k *= ( beta + j) / ( len(V)*beta + i) 
                        i += 1

                if pro_new_k == 0:
                    pro_new_k = sys.float_info.min        
                log_pro.append(pro_new_k)

                sum_pro=sum(log_pro)

                normalized_posterior = [i/sum_pro for i in log_pro]    
                select_k = None
                if iter == (total_iter - 1):
                    select_k = normalized_posterior.index(max(normalized_posterior))

                else:
                    select_k = np.random.choice( (len(Topics)+1) , 1, p=normalized_posterior)[0]  

                if select_k == len(Topics):
                    choose_k = np.max(Topics) + 1
                else:
                    choose_k = Topics[select_k]

                D.add(docID)
                docID_assign_z[docID] = choose_k
                if choose_k not in m_z.keys():
                    m_z[choose_k] = set()
                m_z[choose_k].add(docID)
                for biterm in words:
                    for word in biterm.split(','):
                        if choose_k not in n_w.keys():
                            n_w[choose_k] = 0
                        if choose_k not in n_z.keys():
                            n_z[choose_k] = {}
                        if word not in n_z[choose_k].keys():
                            n_z[choose_k][word] = 0
                        n_z[choose_k][word] += 1
                        n_w[choose_k] += 1
                        V.add(word)
                if choose_k not in Topics:
                    Topics.append(choose_k)

            count_k = []
            for k in Topics:
                if k in m_z.keys() and len(m_z[k]) == 0:
                    m_z.pop(k, None)
                    n_z.pop(k, None)
                    n_w.pop(k, None)
                    count_k.append(k)
            for k in count_k:
                Topics.remove(k)   
        if iter == 9:
            from sklearn.metrics.cluster import normalized_mutual_info_score
            nmi_sample = []
            nmi_result = []
            for key, value in news_labels.items():
                if key < end and key >= start:
                    nmi_sample.append(value)
                    nmi_result.append(docID_assign_z[key])
            NMI_batch[batch_i] = normalized_mutual_info_score(np.array(nmi_sample), np.array(nmi_result))
            topic_batch[batch_i] = len(Topics)
            print("start ",start, " end ",end, "topics: ",len(Topics),"truth: ",len(np.unique(nmi_sample)), " NMI: ",normalized_mutual_info_score(np.array(nmi_sample), np.array(nmi_result)))
    if batch_i != 1:
        start = int(len(corpus) / total_batch) * (batch_i -1) + 1
    

start  0  end  1895 topics:  77 truth:  89  NMI:  0.8174738724577841


KeyboardInterrupt: 

In [3]:
total_batch = 16
NMI_batch = {}
start = 0
end = 0
topic_batch = {}
for batch_i in range(1,17):
    if batch_i == 16:
        end = int(len(corpus))
    else:
        end = int(len(corpus) / total_batch) * batch_i
    
    docID_assign_z = {}
    m_z = {}
    n_z = {}
    n_w = {}
    n_b = {}
    Topics = []
    V = set()
    D = set()
    alpha = 0.3
    beta = 0.002
    NMI_batch[batch_i] = 0
    topic_batch[batch_i] = 0
    
    total_iter = 10
    for iter in range(total_iter):
        for docID in range(start,end):
            words = corpus[docID]
            D.discard(docID)
            if docID in docID_assign_z.keys():
                before_k = docID_assign_z[docID]
                m_z[before_k].discard(docID)
                for biterm in words:
                    for word in biterm.split(','):
                        n_z[before_k][word] -= 1
                        n_w[before_k] -=1
            else:
                before_k = -1
            if len(D) == 0 and len(V) == 0:
                choose_k = 0
                D.add(docID)
                docID_assign_z[docID] = choose_k
                if choose_k not in m_z.keys():
                    m_z[choose_k] = set()
                m_z[choose_k].add(docID)
                for biterm in words:
                    for word in biterm.split(','):
                        if choose_k not in n_w.keys():
                            n_w[choose_k] = 0
                        if choose_k not in n_z.keys():
                            n_z[choose_k] = {}
                        if word not in n_z[choose_k].keys():
                            n_z[choose_k][word] = 0
                        n_z[choose_k][word] += 1
                        n_w[choose_k] += 1
                        V.add(word)
                if choose_k not in Topics:
                    Topics.append(choose_k)
            else:
                log_pro = []
                for k in Topics:
                    pro_k = len(m_z[k])
                    if pro_k != 0:
                        i = 0
                        for biterm in words:
                            each_word = biterm.strip().split(',')
                            for word in each_word:
                                if word not in n_z[k].keys():
                                    n_z[k][word] = 0
                            for j in range(doc_biterm[docID][biterm]):
                                pro_k *= (n_z[k][each_word[0]] + n_z[k][each_word[-1]] + beta + j) / ( n_w[k] + len(V)*beta + i) 
                                i += 1

                    if pro_k == 0:
                        pro_k = sys.float_info.min
                    log_pro.append(pro_k)

                pro_new_k = alpha*(len(D))
                i = 0
                for biterm in words:
                    for j in range(doc_biterm[docID][biterm]):
                        pro_new_k *= ( beta + j) / ( len(V)*beta + i) 
                        i += 1

                if pro_new_k == 0:
                    pro_new_k = sys.float_info.min        
                log_pro.append(pro_new_k)

                sum_pro=sum(log_pro)

                normalized_posterior = [i/sum_pro for i in log_pro]    
                select_k = None
                if iter == (total_iter - 1):
                    select_k = normalized_posterior.index(max(normalized_posterior))

                else:
                    select_k = np.random.choice( (len(Topics)+1) , 1, p=normalized_posterior)[0]  

                if select_k == len(Topics):
                    choose_k = np.max(Topics) + 1
                else:
                    choose_k = Topics[select_k]

                D.add(docID)
                docID_assign_z[docID] = choose_k
                if choose_k not in m_z.keys():
                    m_z[choose_k] = set()
                m_z[choose_k].add(docID)
                for biterm in words:
                    for word in biterm.split(','):
                        if choose_k not in n_w.keys():
                            n_w[choose_k] = 0
                        if choose_k not in n_z.keys():
                            n_z[choose_k] = {}
                        if word not in n_z[choose_k].keys():
                            n_z[choose_k][word] = 0
                        n_z[choose_k][word] += 1
                        n_w[choose_k] += 1
                        V.add(word)
                if choose_k not in Topics:
                    Topics.append(choose_k)

            count_k = []
            for k in Topics:
                if k in m_z.keys() and len(m_z[k]) == 0:
                    m_z.pop(k, None)
                    n_z.pop(k, None)
                    n_w.pop(k, None)
                    count_k.append(k)
            for k in count_k:
                Topics.remove(k)   
        if iter == 9:
            from sklearn.metrics.cluster import normalized_mutual_info_score
            nmi_sample = []
            nmi_result = []
            for key, value in news_labels.items():
                if key < end and key >= start:
                    nmi_sample.append(value)
                    nmi_result.append(docID_assign_z[key])
            NMI_batch[batch_i] = normalized_mutual_info_score(np.array(nmi_sample), np.array(nmi_result))
            topic_batch[batch_i] = len(Topics)
            print("start ",start, " end ",end, "topics: ",len(Topics),"truth: ",len(np.unique(nmi_sample)), " NMI: ",normalized_mutual_info_score(np.array(nmi_sample), np.array(nmi_result)))
    if batch_i != 1:
        start = int(len(corpus) / total_batch) * (batch_i -1) + 1
    

start  0  end  1895 topics:  170 truth:  89  NMI:  0.82200209571536
start  0  end  3790 topics:  220 truth:  98  NMI:  0.823954185001369
start  1896  end  5685 topics:  213 truth:  96  NMI:  0.8199637234493342
start  3791  end  7580 topics:  218 truth:  91  NMI:  0.8110248917572214
start  5686  end  9475 topics:  217 truth:  143  NMI:  0.8315142185043477
start  7581  end  11370 topics:  184 truth:  127  NMI:  0.8715208980672882
start  9476  end  13265 topics:  139 truth:  90  NMI:  0.8950836140923991
start  11371  end  15160 topics:  148 truth:  93  NMI:  0.8636605624730506
start  13266  end  17055 topics:  172 truth:  97  NMI:  0.8481595064881445
start  15161  end  18950 topics:  158 truth:  92  NMI:  0.8740242179908456
start  17056  end  20845 topics:  145 truth:  92  NMI:  0.8882630760417017
start  18951  end  22740 topics:  154 truth:  89  NMI:  0.9008843321539031
start  20846  end  24635 topics:  186 truth:  115  NMI:  0.8583873328536438
start  22741  end  26530 topics:  208 truth

In [5]:
common_keys = []

item_NMI = []
for key,value in sorted(NMI_batch.items(), key=lambda x: x[0]):
    common_keys.append(key)
    item_NMI.append(value)
print(common_keys)

print()

print(item_NMI)
print()

print(np.mean(item_NMI))

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]

[0.82200209571536, 0.823954185001369, 0.8199637234493342, 0.8110248917572214, 0.8315142185043477, 0.8715208980672882, 0.8950836140923991, 0.8636605624730506, 0.8481595064881445, 0.8740242179908456, 0.8882630760417017, 0.9008843321539031, 0.8583873328536438, 0.8290221293591598, 0.806728535675004, 0.7837130812105415]

0.8454941500520822


In [6]:
import timeit
total_batch = 16
NMI_batch = {}
start = 0
end = 0
topic_batch = {}
time_iteration_list = []
for batch_i in range(1,17):
    if batch_i == 16:
        end = int(len(corpus))
    else:
        end = int(len(corpus) / total_batch) * batch_i
    
    docID_assign_z = {}
    m_z = {}
    n_z = {}
    n_w = {}
    n_b = {}
    Topics = []
    V = set()
    D = set()
    alpha = 0.3
    beta = 0.002
    NMI_batch[batch_i] = 0
    topic_batch[batch_i] = 0
    
    time_start = timeit.default_timer()
    total_iter = 10
    for iter in range(total_iter):
        for docID in range(start,end):
            words = corpus[docID]
            D.discard(docID)
            if docID in docID_assign_z.keys():
                before_k = docID_assign_z[docID]
                m_z[before_k].discard(docID)
                for biterm in words:
                    for word in biterm.split(','):
                        n_z[before_k][word] -= 1
                        n_w[before_k] -=1
            else:
                before_k = -1
            if len(D) == 0 and len(V) == 0:
                choose_k = 0
                D.add(docID)
                docID_assign_z[docID] = choose_k
                if choose_k not in m_z.keys():
                    m_z[choose_k] = set()
                m_z[choose_k].add(docID)
                for biterm in words:
                    for word in biterm.split(','):
                        if choose_k not in n_w.keys():
                            n_w[choose_k] = 0
                        if choose_k not in n_z.keys():
                            n_z[choose_k] = {}
                        if word not in n_z[choose_k].keys():
                            n_z[choose_k][word] = 0
                        n_z[choose_k][word] += 1
                        n_w[choose_k] += 1
                        V.add(word)
                if choose_k not in Topics:
                    Topics.append(choose_k)
            else:
                log_pro = []
                for k in Topics:
                    pro_k = len(m_z[k])
                    if pro_k != 0:
                        i = 0
                        for biterm in words:
                            each_word = biterm.strip().split(',')
                            for word in each_word:
                                if word not in n_z[k].keys():
                                    n_z[k][word] = 0
                            for j in range(doc_biterm[docID][biterm]):
                                pro_k *= (n_z[k][each_word[0]] + n_z[k][each_word[-1]] + beta + j) / ( n_w[k] + len(V)*beta + i) 
                                i += 1

                    if pro_k == 0:
                        pro_k = sys.float_info.min
                    log_pro.append(pro_k)

                pro_new_k = alpha*(len(D))
                i = 0
                for biterm in words:
                    for j in range(doc_biterm[docID][biterm]):
                        pro_new_k *= ( beta + j) / ( len(V)*beta + i) 
                        i += 1

                if pro_new_k == 0:
                    pro_new_k = sys.float_info.min        
                log_pro.append(pro_new_k)

                sum_pro=sum(log_pro)

                normalized_posterior = [i/sum_pro for i in log_pro]    
                select_k = None
                if iter == (total_iter - 1):
                    select_k = normalized_posterior.index(max(normalized_posterior))

                else:
                    select_k = np.random.choice( (len(Topics)+1) , 1, p=normalized_posterior)[0]  

                if select_k == len(Topics):
                    choose_k = np.max(Topics) + 1
                else:
                    choose_k = Topics[select_k]

                D.add(docID)
                docID_assign_z[docID] = choose_k
                if choose_k not in m_z.keys():
                    m_z[choose_k] = set()
                m_z[choose_k].add(docID)
                for biterm in words:
                    for word in biterm.split(','):
                        if choose_k not in n_w.keys():
                            n_w[choose_k] = 0
                        if choose_k not in n_z.keys():
                            n_z[choose_k] = {}
                        if word not in n_z[choose_k].keys():
                            n_z[choose_k][word] = 0
                        n_z[choose_k][word] += 1
                        n_w[choose_k] += 1
                        V.add(word)
                if choose_k not in Topics:
                    Topics.append(choose_k)

            count_k = []
            for k in Topics:
                if k in m_z.keys() and len(m_z[k]) == 0:
                    m_z.pop(k, None)
                    n_z.pop(k, None)
                    n_w.pop(k, None)
                    count_k.append(k)
            for k in count_k:
                Topics.remove(k)   
        if iter == 9:
            from sklearn.metrics.cluster import normalized_mutual_info_score
            nmi_sample = []
            nmi_result = []
            for key, value in news_labels.items():
                if key < end and key >= start:
                    nmi_sample.append(value)
                    nmi_result.append(docID_assign_z[key])
            NMI_batch[batch_i] = normalized_mutual_info_score(np.array(nmi_sample), np.array(nmi_result))
            topic_batch[batch_i] = len(Topics)
            print("start ",start, " end ",end, "topics: ",len(Topics),"truth: ",len(np.unique(nmi_sample)), " NMI: ",normalized_mutual_info_score(np.array(nmi_sample), np.array(nmi_result)))
    if batch_i != 1:
        start = int(len(corpus) / total_batch) * (batch_i -1) + 1
    time_stop = timeit.default_timer()
    time_iteration_list.append(time_stop-time_start)
    print(time_iteration_list)

start  0  end  1895 topics:  177 truth:  89  NMI:  0.8223536740332866
[216.0135057865846]
start  0  end  3790 topics:  224 truth:  98  NMI:  0.8177250805428806
[216.0135057865846, 509.70495043912064]
start  1896  end  5685 topics:  220 truth:  96  NMI:  0.8272888867666193
[216.0135057865846, 509.70495043912064, 487.1394209779729]
start  3791  end  7580 topics:  205 truth:  91  NMI:  0.8151190449987157
[216.0135057865846, 509.70495043912064, 487.1394209779729, 443.31294940791076]
start  5686  end  9475 topics:  224 truth:  143  NMI:  0.8258527312415648
[216.0135057865846, 509.70495043912064, 487.1394209779729, 443.31294940791076, 693.9138134369191]
start  7581  end  11370 topics:  175 truth:  127  NMI:  0.8684109674328697
[216.0135057865846, 509.70495043912064, 487.1394209779729, 443.31294940791076, 693.9138134369191, 592.1550351563897]
start  9476  end  13265 topics:  145 truth:  90  NMI:  0.8898665292074673
[216.0135057865846, 509.70495043912064, 487.1394209779729, 443.31294940791076,

In [ ]:
0.8574291521104326

In [18]:
common_keys = []

item_NMI = []
for key,value in sorted(NMI_batch.items(), key=lambda x: x[0]):
    common_keys.append(key)
    item_NMI.append(value)
print(common_keys)

print()

print(item_NMI)
print()

print(np.mean(item_NMI))

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]

[0.8323176994066535, 0.8321619534393847, 0.8260085749718509, 0.8363347268544267, 0.8466603284526395, 0.8880658911998899, 0.9046168269190166, 0.8685988542157779, 0.8631199199499224, 0.883549612851989, 0.8928763264307683, 0.9066801013701994, 0.8779024429094671, 0.8429377961401533, 0.828917548973653, 0.7881178296811294]

0.8574291521104326


In [17]:
common_keys = []

item_NMI = []
for key,value in sorted(NMI_batch.items(), key=lambda x: x[0]):
    common_keys.append(key)
    item_NMI.append(value)
print(common_keys)

print()

print(item_NMI)
print()

print(np.mean(item_NMI))

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]

[0.8323176994066535, 0.8321619534393847, 0.8260085749718509, 0.8363347268544267, 0.8466603284526395, 0.8880658911998899, 0.9046168269190166, 0.8685988542157779, 0.8631199199499224, 0.883549612851989, 0.8928763264307683, 0.9066801013701994, 0.8779024429094671, 0.8429377961401533, 0.828917548973653, 0.7881178296811294]

0.8574291521104326


In [20]:
print(NMI_batch)
print(beta)
print(NMI_batch.keys())
print(NMI_batch.values())

{1: 0.8278218254942188, 2: 0.8301163585429583, 3: 0.8276996464223356, 4: 0.8266799384278927, 5: 0.8361987253008136, 6: 0.8918132625586427, 7: 0.9143589365596025, 8: 0.8755057317179911, 9: 0.8761813446904034, 10: 0.8937849598731826, 11: 0.9038236974191605, 12: 0.9076669340448583, 13: 0.8717577659131356, 14: 0.8359866418427073, 15: 0.8274625028703796, 16: 0.7925454624242205}
0.02
dict_keys([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16])
dict_values([0.8278218254942188, 0.8301163585429583, 0.8276996464223356, 0.8266799384278927, 0.8361987253008136, 0.8918132625586427, 0.9143589365596025, 0.8755057317179911, 0.8761813446904034, 0.8937849598731826, 0.9038236974191605, 0.9076669340448583, 0.8717577659131356, 0.8359866418427073, 0.8274625028703796, 0.7925454624242205])


In [25]:
nmi_batch = {}
topic_batch = {}
start = 0
end = 0
total_batch = None
if len(corpus) % 500 == 0:
    total_batch = int(len(corpus) / 500)
else:
    total_batch = int(len(corpus) / 500) +1

for batch in range(total_batch):
    docID_assign_z = {}
    m_z = {}
    n_z = {}
    n_w = {}
    n_b = {}
    Topics = []
    V = set()
    D = set()
    alpha = 0.3
    beta = 0.002
    end = 500 * (batch + 1)
    if end > len(corpus):
        end = len(corpus)
    nmi_batch[batch] = 0
    topic_batch[batch] = 0
    
    total_iter = 10
    for iter in range(total_iter):
        for docID in range(start,end):
            words = corpus[docID]
            D.discard(docID)
            if docID in docID_assign_z.keys():
                before_k = docID_assign_z[docID]
                m_z[before_k].discard(docID)
                for biterm in words:
                    for word in biterm.split(','):
                        n_z[before_k][word] -= 1
                        n_w[before_k] -=1
            else:
                before_k = -1
            if len(D) == 0 and len(V) == 0:
                choose_k = 0
                D.add(docID)
                docID_assign_z[docID] = choose_k
                if choose_k not in m_z.keys():
                    m_z[choose_k] = set()
                m_z[choose_k].add(docID)
                for biterm in words:
                    for word in biterm.split(','):
                        if choose_k not in n_w.keys():
                            n_w[choose_k] = 0
                        if choose_k not in n_z.keys():
                            n_z[choose_k] = {}
                        if word not in n_z[choose_k].keys():
                            n_z[choose_k][word] = 0
                        n_z[choose_k][word] += 1
                        n_w[choose_k] += 1
                        V.add(word)
                if choose_k not in Topics:
                    Topics.append(choose_k)
            else:
                log_pro = []
                for k in Topics:
                    pro_k = len(m_z[k])
                    if pro_k != 0:
                        i = 0
                        for biterm in words:
                            each_word = biterm.strip().split(',')
                            for word in each_word:
                                if word not in n_z[k].keys():
                                    n_z[k][word] = 0
                            for j in range(doc_biterm[docID][biterm]):
                                pro_k *= (n_z[k][each_word[0]] + n_z[k][each_word[-1]] + beta + j) / ( n_w[k] + len(V)*beta + i) 
                                i += 1

                    if pro_k == 0:
                        pro_k = sys.float_info.min
                    log_pro.append(pro_k)

                pro_new_k = alpha*(len(D))
                i = 0
                for biterm in words:
                    for j in range(doc_biterm[docID][biterm]):
                        pro_new_k *= ( beta + j) / ( len(V)*beta + i) 
                        i += 1

                if pro_new_k == 0:
                    pro_new_k = sys.float_info.min        
                log_pro.append(pro_new_k)

                sum_pro=sum(log_pro)

                normalized_posterior = [i/sum_pro for i in log_pro]    
                select_k = None
                if iter == (total_iter - 1):
                    select_k = normalized_posterior.index(max(normalized_posterior))

                else:
                    select_k = np.random.choice( (len(Topics)+1) , 1, p=normalized_posterior)[0]  

                if select_k == len(Topics):
                    choose_k = np.max(Topics) + 1
                else:
                    choose_k = Topics[select_k]

                D.add(docID)
                docID_assign_z[docID] = choose_k
                if choose_k not in m_z.keys():
                    m_z[choose_k] = set()
                m_z[choose_k].add(docID)
                for biterm in words:
                    for word in biterm.split(','):
                        if choose_k not in n_w.keys():
                            n_w[choose_k] = 0
                        if choose_k not in n_z.keys():
                            n_z[choose_k] = {}
                        if word not in n_z[choose_k].keys():
                            n_z[choose_k][word] = 0
                        n_z[choose_k][word] += 1
                        n_w[choose_k] += 1
                        V.add(word)
                if choose_k not in Topics:
                    Topics.append(choose_k)

            count_k = []
            for k in Topics:
                if k in m_z.keys() and len(m_z[k]) == 0:
                    m_z.pop(k, None)
                    n_z.pop(k, None)
                    n_w.pop(k, None)
                    count_k.append(k)
            for k in count_k:
                Topics.remove(k)   
        if iter == 9:
            from sklearn.metrics.cluster import normalized_mutual_info_score
            nmi_sample = []
            nmi_result = []
            for key, value in news_labels.items():
                if key < end and key >= start:
                    nmi_sample.append(value)
                    nmi_result.append(docID_assign_z[key])
            nmi_batch[batch] = normalized_mutual_info_score(np.array(nmi_sample), np.array(nmi_result))
            topic_batch[batch] = len(Topics)
            print("start ",start, " end ",end, "topics: ",len(Topics),"truth: ",len(np.unique(nmi_sample)), " NMI: ",normalized_mutual_info_score(np.array(nmi_sample), np.array(nmi_result)))
    start = end 
 
print(nmi_batch.keys())
print(nmi_batch.values())

start  0  end  500 topics:  103 truth:  68  NMI:  0.8474288340533979
start  500  end  1000 topics:  92 truth:  72  NMI:  0.8534737854841058
start  1000  end  1500 topics:  98 truth:  69  NMI:  0.8428782378743177
start  1500  end  2000 topics:  98 truth:  69  NMI:  0.856304660474624
start  2000  end  2500 topics:  98 truth:  68  NMI:  0.8523640295032716
start  2500  end  3000 topics:  86 truth:  68  NMI:  0.8860828017208211
start  3000  end  3500 topics:  94 truth:  70  NMI:  0.8604437694483016
start  3500  end  4000 topics:  92 truth:  72  NMI:  0.848645947842599
start  4000  end  4500 topics:  86 truth:  72  NMI:  0.8691456112814188
start  4500  end  5000 topics:  92 truth:  68  NMI:  0.8494774982467017
start  5000  end  5500 topics:  92 truth:  58  NMI:  0.8848503817879018
start  5500  end  6000 topics:  99 truth:  60  NMI:  0.8360336655931694
start  6000  end  6500 topics:  90 truth:  61  NMI:  0.8215379347028716
start  6500  end  7000 topics:  97 truth:  54  NMI:  0.825404231119372

In [40]:
total_batch = 16
NMI_batch = {}
start = 0
end = 0
topic_batch = {}
for batch_i in range(1,17):
    if batch_i == 16:
        end = int(len(corpus))
    else:
        end = int(len(corpus) / total_batch) * batch_i
    
    docID_assign_z = {}
    m_z = {}
    n_z = {}
    n_w = {}
    n_b = {}
    Topics = []
    V = set()
    D = set()
    alpha = 0.3
    beta = 0.003
    NMI_batch[batch_i] = 0
    topic_batch[batch_i] = 0
    
    total_iter = 10
    for iter in range(total_iter):
        for docID in range(start,end):
            words = corpus[docID]
            D.discard(docID)
            if docID in docID_assign_z.keys():
                before_k = docID_assign_z[docID]
                m_z[before_k].discard(docID)
                for biterm in words:
                    for word in biterm.split(','):
                        n_z[before_k][word] -= 1
                        n_w[before_k] -=1
            else:
                before_k = -1
            if len(D) == 0 and len(V) == 0:
                choose_k = 0
                D.add(docID)
                docID_assign_z[docID] = choose_k
                if choose_k not in m_z.keys():
                    m_z[choose_k] = set()
                m_z[choose_k].add(docID)
                for biterm in words:
                    for word in biterm.split(','):
                        if choose_k not in n_w.keys():
                            n_w[choose_k] = 0
                        if choose_k not in n_z.keys():
                            n_z[choose_k] = {}
                        if word not in n_z[choose_k].keys():
                            n_z[choose_k][word] = 0
                        n_z[choose_k][word] += 1
                        n_w[choose_k] += 1
                        V.add(word)
                if choose_k not in Topics:
                    Topics.append(choose_k)
            else:
                log_pro = []
                for k in Topics:
                    pro_k = len(m_z[k])
                    if pro_k != 0:
                        i = 0
                        for biterm in words:
                            each_word = biterm.strip().split(',')
                            for word in each_word:
                                if word not in n_z[k].keys():
                                    n_z[k][word] = 0
                            for j in range(doc_biterm[docID][biterm]):
                                pro_k *= (n_z[k][each_word[0]] + n_z[k][each_word[-1]] + beta + j) / ( n_w[k] + len(V)*beta + i) 
                                i += 1

                    if pro_k == 0:
                        pro_k = sys.float_info.min
                    log_pro.append(pro_k)

                pro_new_k = alpha*(len(D))
                i = 0
                for biterm in words:
                    for j in range(doc_biterm[docID][biterm]):
                        pro_new_k *= ( beta + j) / ( len(V)*beta + i) 
                        i += 1

                if pro_new_k == 0:
                    pro_new_k = sys.float_info.min        
                log_pro.append(pro_new_k)

                sum_pro=sum(log_pro)

                normalized_posterior = [i/sum_pro for i in log_pro]    
                select_k = None
                if iter == (total_iter - 1):
                    select_k = normalized_posterior.index(max(normalized_posterior))

                else:
                    select_k = np.random.choice( (len(Topics)+1) , 1, p=normalized_posterior)[0]  

                if select_k == len(Topics):
                    choose_k = np.max(Topics) + 1
                else:
                    choose_k = Topics[select_k]

                D.add(docID)
                docID_assign_z[docID] = choose_k
                if choose_k not in m_z.keys():
                    m_z[choose_k] = set()
                m_z[choose_k].add(docID)
                for biterm in words:
                    for word in biterm.split(','):
                        if choose_k not in n_w.keys():
                            n_w[choose_k] = 0
                        if choose_k not in n_z.keys():
                            n_z[choose_k] = {}
                        if word not in n_z[choose_k].keys():
                            n_z[choose_k][word] = 0
                        n_z[choose_k][word] += 1
                        n_w[choose_k] += 1
                        V.add(word)
                if choose_k not in Topics:
                    Topics.append(choose_k)

            count_k = []
            for k in Topics:
                if k in m_z.keys() and len(m_z[k]) == 0:
                    m_z.pop(k, None)
                    n_z.pop(k, None)
                    n_w.pop(k, None)
                    count_k.append(k)
            for k in count_k:
                Topics.remove(k)   
        print("iter ",iter, " topic: ", len(Topics))
        if iter == 9:
            from sklearn.metrics.cluster import normalized_mutual_info_score
            nmi_sample = []
            nmi_result = []
            for key, value in news_labels.items():
                if key < end and key >= start:
                    nmi_sample.append(value)
                    nmi_result.append(docID_assign_z[key])
            NMI_batch[batch_i] = normalized_mutual_info_score(np.array(nmi_sample), np.array(nmi_result))
            topic_batch[batch_i] = len(Topics)
            print("start ",start, " end ",end, "topics: ",len(Topics),"truth: ",len(np.unique(nmi_sample)), " NMI: ",normalized_mutual_info_score(np.array(nmi_sample), np.array(nmi_result)))
    if batch_i != 1:
        start = int(len(corpus) / total_batch) * (batch_i -1) + 1
    

iter  0  topic:  139
iter  1  topic:  156
iter  2  topic:  165
iter  3  topic:  165
iter  4  topic:  172
iter  5  topic:  173
iter  6  topic:  169
iter  7  topic:  171
iter  8  topic:  177
iter  9  topic:  170
start  0  end  1895 topics:  170 truth:  89  NMI:  0.8234967923953466
iter  0  topic:  175
iter  1  topic:  186
iter  2  topic:  194
iter  3  topic:  203
iter  4  topic:  205
iter  5  topic:  216
iter  6  topic:  215
iter  7  topic:  208
iter  8  topic:  207
iter  9  topic:  203
start  0  end  3790 topics:  203 truth:  98  NMI:  0.8311417812056228
iter  0  topic:  173
iter  1  topic:  195
iter  2  topic:  202
iter  3  topic:  204
iter  4  topic:  205
iter  5  topic:  210
iter  6  topic:  213
iter  7  topic:  218
iter  8  topic:  217
iter  9  topic:  216
start  1896  end  5685 topics:  216 truth:  96  NMI:  0.823754584299913
iter  0  topic:  169
iter  1  topic:  182
iter  2  topic:  186
iter  3  topic:  193
iter  4  topic:  204
iter  5  topic:  206
iter  6  topic:  211
iter  7  to

In [41]:
common_keys = []

item_NMI = []
for key,value in sorted(NMI_batch.items(), key=lambda x: x[0]):
    common_keys.append(key)
    item_NMI.append(value)
print(common_keys)

print()

print(item_NMI)
print(np.mean(item_NMI))

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]

[0.8234967923953466, 0.8311417812056228, 0.823754584299913, 0.8224176781096412, 0.8342011829877818, 0.8807817337139942, 0.892479261642998, 0.8593393266744448, 0.8513263110415927, 0.8706397677696075, 0.8869756524583201, 0.887975516312373, 0.8750403623475929, 0.8292225144619587, 0.795579346957113, 0.7820984109771273]
0.8466543889597142


In [36]:
common_keys = []

item_NMI = []
for key,value in sorted(NMI_batch.items(), key=lambda x: x[0]):
    common_keys.append(key)
    item_NMI.append(value)
print(common_keys)

print()

print(item_NMI)
print(np.mean(item_NMI))

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]

[0.8298927392436527, 0.8269852334907385, 0.8275801293034326, 0.8325725508167305, 0.8296941181191704, 0.8858025131592013, 0.9112544715777952, 0.882279072937081, 0.8625284434677941, 0.8870335110637954, 0.9027410653955591, 0.8983125116090817, 0.8757577934488942, 0.8330259394498267, 0.8076724003551009, 0.7923083439379209]
0.855340052335986


In [7]:
alpha_NMI = {}
alpha_topics = {}
for alpha in np.arange(0.1,1.1,0.1):
    alpha = np.around(alpha,decimals=1)
    
    total_batch = 16
    NMI_batch = {}
    start = 0
    end = 0
    topic_batch = {}
    time_iteration_list = []
    for batch_i in range(1,17):
        if batch_i == 16:
            end = int(len(corpus))
        else:
            end = int(len(corpus) / total_batch) * batch_i

        docID_assign_z = {}
        m_z = {}
        n_z = {}
        n_w = {}
        n_b = {}
        Topics = []
        V = set()
        D = set()
        #alpha = 0.6
        beta = 0.02
        NMI_batch[batch_i] = 0
        topic_batch[batch_i] = 0

        time_start = timeit.default_timer()
        total_iter = 10
        for iter in range(total_iter):
            for docID in range(start,end):
                words = corpus[docID]
                D.discard(docID)
                if docID in docID_assign_z.keys():
                    before_k = docID_assign_z[docID]
                    m_z[before_k].discard(docID)
                    for biterm in words:
                        for word in biterm.split(','):
                            n_z[before_k][word] -= 1
                            n_w[before_k] -=1
                else:
                    before_k = -1
                if len(D) == 0 and len(V) == 0:
                    choose_k = 0
                    D.add(docID)
                    docID_assign_z[docID] = choose_k
                    if choose_k not in m_z.keys():
                        m_z[choose_k] = set()
                    m_z[choose_k].add(docID)
                    for biterm in words:
                        for word in biterm.split(','):
                            if choose_k not in n_w.keys():
                                n_w[choose_k] = 0
                            if choose_k not in n_z.keys():
                                n_z[choose_k] = {}
                            if word not in n_z[choose_k].keys():
                                n_z[choose_k][word] = 0
                            n_z[choose_k][word] += 1
                            n_w[choose_k] += 1
                            V.add(word)
                    if choose_k not in Topics:
                        Topics.append(choose_k)
                else:
                    log_pro = []
                    for k in Topics:
                        pro_k = len(m_z[k])
                        if pro_k != 0:
                            i = 0
                            for biterm in words:
                                each_word = biterm.strip().split(',')
                                for word in each_word:
                                    if word not in n_z[k].keys():
                                        n_z[k][word] = 0
                                for j in range(doc_biterm[docID][biterm]):
                                    pro_k *= (n_z[k][each_word[0]] + n_z[k][each_word[-1]] + beta + j) / ( n_w[k] + len(V)*beta + i) 
                                    i += 1

                        if pro_k == 0:
                            pro_k = sys.float_info.min
                        log_pro.append(pro_k)

                    pro_new_k = alpha*(len(D))
                    i = 0
                    for biterm in words:
                        for j in range(doc_biterm[docID][biterm]):
                            pro_new_k *= ( beta + j) / ( len(V)*beta + i) 
                            i += 1

                    if pro_new_k == 0:
                        pro_new_k = sys.float_info.min        
                    log_pro.append(pro_new_k)

                    sum_pro=sum(log_pro)

                    normalized_posterior = [i/sum_pro for i in log_pro]    
                    select_k = None
                    if iter == (total_iter - 1):
                        select_k = normalized_posterior.index(max(normalized_posterior))

                    else:
                        select_k = np.random.choice( (len(Topics)+1) , 1, p=normalized_posterior)[0]  

                    if select_k == len(Topics):
                        choose_k = np.max(Topics) + 1
                    else:
                        choose_k = Topics[select_k]

                    D.add(docID)
                    docID_assign_z[docID] = choose_k
                    if choose_k not in m_z.keys():
                        m_z[choose_k] = set()
                    m_z[choose_k].add(docID)
                    for biterm in words:
                        for word in biterm.split(','):
                            if choose_k not in n_w.keys():
                                n_w[choose_k] = 0
                            if choose_k not in n_z.keys():
                                n_z[choose_k] = {}
                            if word not in n_z[choose_k].keys():
                                n_z[choose_k][word] = 0
                            n_z[choose_k][word] += 1
                            n_w[choose_k] += 1
                            V.add(word)
                    if choose_k not in Topics:
                        Topics.append(choose_k)

                count_k = []
                for k in Topics:
                    if k in m_z.keys() and len(m_z[k]) == 0:
                        m_z.pop(k, None)
                        n_z.pop(k, None)
                        n_w.pop(k, None)
                        count_k.append(k)
                for k in count_k:
                    Topics.remove(k)   
            if iter == 9:
                from sklearn.metrics.cluster import normalized_mutual_info_score
                nmi_sample = []
                nmi_result = []
                for key, value in news_labels.items():
                    if key < end and key >= start:
                        nmi_sample.append(value)
                        nmi_result.append(docID_assign_z[key])
                NMI_batch[batch_i] = normalized_mutual_info_score(np.array(nmi_sample), np.array(nmi_result))
                topic_batch[batch_i] = len(Topics)
                #print("start ",start, " end ",end, "topics: ",len(Topics),"truth: ",len(np.unique(nmi_sample)), " NMI: ",normalized_mutual_info_score(np.array(nmi_sample), np.array(nmi_result)))
        if batch_i != 1:
            start = int(len(corpus) / total_batch) * (batch_i -1) + 1
    
    alpha_NMI[alpha] = np.mean(list(NMI_batch.values()))
    alpha_topics[alpha] = int(np.mean(list(topic_batch.values())))
    print("alpha ",alpha, " NMI: ", alpha_NMI[alpha])
    print("alpha ",alpha, " Topics: ", alpha_topics[alpha])

alpha  0.1  NMI:  0.857268408981715
alpha  0.1  Topics:  123
alpha  0.2  NMI:  0.8592148717596345
alpha  0.2  Topics:  126
alpha  0.3  NMI:  0.8575133376480126
alpha  0.3  Topics:  127
alpha  0.4  NMI:  0.8562499910589776
alpha  0.4  Topics:  131
alpha  0.5  NMI:  0.857533270578133
alpha  0.5  Topics:  135
alpha  0.6  NMI:  0.8575866690811085
alpha  0.6  Topics:  136
alpha  0.7  NMI:  0.8582720702474204
alpha  0.7  Topics:  139
alpha  0.8  NMI:  0.8570571645878108
alpha  0.8  Topics:  139
alpha  0.9  NMI:  0.8573575831418399
alpha  0.9  Topics:  147
alpha  1.0  NMI:  0.8579458438287924
alpha  1.0  Topics:  147


In [8]:
print(alpha_NMI.values())
print(alpha_topics.values())

dict_values([0.857268408981715, 0.8592148717596345, 0.8575133376480126, 0.8562499910589776, 0.857533270578133, 0.8575866690811085, 0.8582720702474204, 0.8570571645878108, 0.8573575831418399, 0.8579458438287924])
dict_values([123, 126, 127, 131, 135, 136, 139, 139, 147, 147])
